# 信号空间
```{epigraph}
道可道非常道，名可名非常名。无名天地之始，有名万物之母。故常无欲以观其妙，常有欲以观其徼。此两者，同出而异名，同谓之玄，玄之又玄，众妙之门。

--老子
```

万千世界起源于基础的规律与法则。这些规律与法则是世界与生俱有的，不是能够被人们所定义的。天地起源于“无”，而“有”产生了万物。

因此我们应该客观地去观察其细节，而这种客观性是在主观的结构观察中进行。

有与无、客观与主观就是一种不可言的基础法则，非常玄妙，是我们认识世界，以及行事的门道。

## 分析流程之结构模型
信号处理就是对一个给定的信号，应用一系列的分析方法，进行分析。虽然分析的方案与细节因项目的性质与目的而异，但是分析流程在结构上有一定的规律。这里我们希望能回答以下的问题：
- 结构的范畴。
- 分析过程的共性。

### 范畴
#### 算法范畴
算法分四类：全局分析，局部分析，自适应分析和自学习性分析。全局分析和局部分析共享基础分析算法。总体结构归纳如下：
- 基础分析方法：傅里叶分析（尤其是余弦波和正弦波分析），卷积，小波分析等。
- 局部分析：对局部信号两端有两类不同的处理
  - 直接截取：对应与block transforms
  - 光滑延伸：对应与lap transforms
- 自适应分析：建立在多层次局部分析的基础上，从不同合法的组合中寻找最佳组合。
- 自学习性：建立在多层局部分析的基础上，也有可能依赖于自适应分析。

#### 信号范畴
对信号的范畴的界定归结为有限化和离散化。
- *时间域*：我们定义自然数$\mathbb{N}$为时间域的空间。也就是说进入这个模型的型号都被认为其时间域是$\mathbb N$，而且是按序排列。至于时间域的真实意义不在这个模型之内。
- *值域*：原始信号由一列数组构成，数组的元素为复数，这个我们称之为*0-空间*。为了满足局部分析，我们还需要引入两个空间，都是序列，区分在于序列的元素。对应与block transform，元素为数组，代表所截取的一段信号，这个空间我们称之为*1-空间*。因此，*1-空间*数组序列构成。对应与lap transforms，序列的元素是一个三体，三体中的每个子元素是一个数组，三体中的左元素是左边延伸的数组，右边是右边延伸的数组，中间则是由直接截取的局部信号。也就是说，这个空间是由三体序列构成，我们称之为*3-空间*。

### 分析过程的共性
#### 特征：
- 字面量数组是基础的分析单元，这个基础单元就是*0-空间*。分析算法最大的特点就是用基础分析方法反复应用在以*0-空间*为数据类型的元素上。
- 类一局部分析：这对应block transform。将类型为0-空间的原始信号截取$n$段，生成1-空间，空间内每个元素以0-空间为类型。遍历1-空间的元素，在遍历的过程中，将基础分析方法应用于每个元素，方法的输出也是0-空间类。
- 全局分析：我们把全局分析看成局部分析的特例。就是1-空间只含有一个以0-空间为类型的元素。
- 类二局部分析：这类分析对应lap transforms。Lap transform的单元函数（也就是作用于一个局部信号的函数）的输入与输出都是三体0-空间：（0-空间，0-空间，0-空间)。Lap transform就是运用单元函数遍历3-空间。

#### 容器（containers）与环境（contexts）
容器和环境的概念是对循环操作过程进行建模而产生。一个循环操作过程可以分解为三个模块：载体，基本元素，对基本元素的操作。容器就是载体，它是基本元素的一个集合，重复的操作是将操作函数作用于基础元素从而产生了新的基础元素。通过遍历我们获得了新的基本元素的集合，也就是新的容器。而这种操作行为我们看成给容器赋予了一个工作环境。

不同的性质的行为决定了不同的环境。行为的性质可以分为同构性与异构性。对多重循环而言，又可以分为独立性与依赖性。

##### 同构与异构
循环的操作就是将一个容器映射到另外一个容器，如果这两个容器是同一个结构，我们称之为同构，否则就是异构。比如函数`float`就是将基础元素为整数（类型为`int`）映射到相应的浮点数（类型为`float`）。一个`list`含有整数的容器在函数`float`的循环操作下变成了含有浮点数的`list`容器，这是一个同构操作。因为容器的结构没有变化，都是含有同样个数的`list`。如果这个操作函数不是`float`，而是将一个整数变成两个整数，那么这个循环操作行为就不是同构的，因为新的容器跟原来的容器含有不同的个数的基础元素。如果这个操作函数是将一个整数变成含有两个整数的一个`list`，那么这个循环操作又是同构的。

##### 独立与依赖
在多重循环中，有三个不同的行为，独立性，依赖性和嵌套性。
- 独立性是指每个循环可以独立进行。
- 依赖性是指当期循环依赖于上一个循环的结果。
- 嵌套式是指多重循环以嵌套的形式进行，也就是上一个循环要依赖于子循环结束才能进行下一个对基础元素的操作。

除了嵌套式外，函数式程式对这些不同环境有明确的定义。对同构，依赖性多循环，Functor是主要工具，Monad也可以胜任。对同构，独立多循环，Applicative是主要工具，Functor, Monad也是可以胜任。对异构，依赖性多循环，Monad是主要工具。

如何对嵌套式进行建模是一个研究课题。我们在自适应性算法实现的时候，会碰到这样的课题，我们解决的方案是采用二叉树的数据结构结合迭代式环境来实现。

## 模型描述
从数学的角度，我们定义了三个信号空间：$\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K~$和$~\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$，分别对应0-空间，1-空间和3-空间。

从范畴（category）学的角度，信号空间可以定义为$\mathfrak{C}=\{\mathfrak{F}_1,\mathfrak{F}_3\}$，即信号空间是范畴$\mathfrak C$，其中含有$\mathfrak{F}_1$与$\mathfrak{F}_3$两个对象（object）。时间域的信息隐含在对象的指针（index）中。一个具体的分析方法本质上就是信号空间$\mathfrak C$的态射（morphism）。我们考虑的态射有一下两类：
- 自同态（endomorphism）：$\mathfrak F_i\longrightarrow\mathfrak F_i,\quad i\in\{1,3\}$，大多数的算法都为自同态。
- 跨对象态射：$\mathfrak F_i\longrightarrow\mathfrak F_j,\quad i,j\in\{1,3\}$，lap transform的逆变换就是从$\mathfrak F_3$到$\mathfrak F_1$的跨对象态射。

在这个模型下，基础算法不是态射，因为它们是$\mathfrak F_i\rightarrow\mathfrak F_i,\quad i\in\{0,1\}$或$\mathfrak F_i\rightarrow\mathfrak F_j,\quad i,j=\{0,3\}$的映射。信号空间的态射却是由这些基础算法提升（lifting）而来。从这个角度上看，上面所阐述的一个环境，就是一种提升的方法。

这个模型勾画了我们业务逻辑的本质，但过于抽象，为了方便实现，我们再定义两个工作范畴：$\mathfrak C_i=\{\mathfrak F_i\},\quad i\in\{1,3\}$。这两个范畴在我们的程序实现上对应与上述的环境。一个具体的分析方法就是这两个范畴之间的映射（$\mathfrak C_i$到$\mathfrak C_i$或$\mathfrak C_i$到$\mathfrak C_j$的映射）。

### 模型定义
综上所述，信号空间模型可以定义为：
- 三个基础集合：$\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K~$和$~\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$。
- 两个范畴：$\mathfrak C_1=\{\mathfrak F_1\}$和$\mathfrak C_3=\{\mathfrak F_3\}$。
- 时间域隐含在集合元素的指针中。
- 分析方法为两个范畴之间的态射。
- 基础算法（简称算法）为基础集合之间的映射。
- 态射由算法提升而来，而这种提升的手段是通过对两个信号范畴的实现而获得，每一个特定的实现就是一种特定的环境，或者说环境就是对信号范畴在程序体中的表达。

## 模型的实现
以上模型的实现涵括以下内容：
- 信号空间的数据类型
- 信号空间环境与态射（算法的提升）
- 算法
- 态射

### 基础信号空间的数据类型
数据类型用type aliases来定义。对基础信号空间而言，这样可以利用`list`的诸多功能，尤其是`Iterable`和`Sequence`的功能。

- `signum`: 定义了0-空间的元素数据类型为字面量`int`, `float`, `complex`，加入`str`是将信号空间拓广到文本分析，`None`与`tuple[()]`则是技术上对空集处理的可能需求。
- `sig_0`：`list[signum]`，对应0-空间。
- `sig_1`: `list[sig_0]`，对应1-空间。
- `sig_3`: `list[(sig_0,sig_0,sig_0)]`，对应3-空间。

In [1]:
#from Signal.Types import signum, sig_0, sig_1, sig_3
import numpy as np
import numpy.typing as npt

signum=int|float|complex|str|None|tuple[()]
#signum=int|float|complex|tuple[()]
sig_0=list[signum]|npt.NDArray
sig_1=list[sig_0]
sig_3=list[tuple[sig_0,sig_0,sig_0]]

### 信号环境
这是对信号范畴的实现。环境包括两大类，计算环境（computational contexts）和迭代环境（iterator contexts）。

计算环境有：
- `Sigor`: `ListFunctor`约束于0-空间，1-空间和3-空间。
- `Sigive`：`ListApplicative`约束于0-空间，1-空间和3-空间。
- `SigMonad`：`ListMonad`约束于0-空间，1-空间和3-空间。

迭代环境：
- `SigFoldable`：`ListFoldable`约束于0-空间，1-空间和3-空间。
- `SigTraversable`：`ListTraversable`约束于0-空间，1-空间和3-空间。

其中`SigTraversable`还没实现。还没实现的计算环境是`SigStateMonad`，一个成熟的用于有限状态机的环境。

由于python的函数式功能散布在语言的各个部分，`map`，`reduce`，`filter`，`iter`，`itertools`，`functools`, `operator`，根据我们的需求我们有系统地对`list`容器赋予了一系列的环境。这一系列的实现放在`Contexts`程序包里，主要的手段就是`generator comprehension`，`abc`和`typing`。

In [2]:
#from Signal.Contexts import Sigor, Sigive, SigMonad, SigLiftA2

from Contexts.Functor import ListFunctor
from Contexts import Functor
from Contexts.Applicative import ListApplicative
from Contexts import Applicative
from Contexts.Monad import ListMonad
from Contexts import Monad
from Contexts.Foldable import ListFoldable
from Contexts import Foldable

from Signal.Types import sig_0

Sigor=ListFunctor[signum|sig_0|tuple[sig_0,sig_0,sig_0]]
Sigive=ListApplicative[signum|sig_0|tuple[sig_0,sig_0,sig_0]]
SigMonad=ListMonad[signum|sig_0|tuple[sig_0,sig_0,sig_0]]
SigLiftA2=Applicative.liftA2
SigFoldable=ListFoldable[signum|sig_0|tuple[sig_0,sig_0,sig_0]]


#### 计算环境

##### `Sigor`: 信号范畴的Functor环境
`Sigor`的关键功能就是`fmap`，是算法（`fmap`的输入参数）的提升器。`Sigor`适合同构，依赖性多循环的操作。以下案例从态射的角度，就是将`float`和`str`两个算法提升到态射层面，把原来的`sig_0[int]`映射为`sig_0[float]`，然后映射成`sig_0[str]`。从循环的角度，依次完成了两个有依赖的循环。

In [3]:
a=[x for x in range(1,9)]
s=Sigor(a)
s.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0']

In [4]:
a=np.array([x for x in range(1,9)])
s=Sigor(a)
s.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0']

`Sigor`还有一个很有意思的简单函数`Functor.const(a,b)：return a`和一个建立在这个函数之上的类方法`mrb`(map-replaced by，在Haskell语言中是算子`<$`)，其功能是将输入参数替换掉容器的每一个元素。`mrb`是通过`fmap.const`来实现，这是一个很有启发的技巧。下面的案例就是将原来的数组`[1,2,3,4,5,6,7,8]`中的每一个元素替换成`[1,2]`:

In [5]:
s.mrb([1,2])

[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]

In [6]:
type(s.mrb([1,2]))

Contexts.Functor.ListFunctor

等价于以下操作：

In [7]:
s.fmap(lambda _: [1,2])

[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]

虽然`ListFunctor`是从`list`继承而来，但是`list`的类方法的返回值还是`list`。比如算子`+`:

In [8]:
type(s+s)

list

我们希望`s+s`还是`ListFunctor`。幸好在函数式程式里有Monoid的环境，这个环境制定了三个法则`mempty`，`mappend`和`mconcat`，分别代表空集，附加和concat。这些都是容器的关键概念和工具。Monoid的基本定义在程序包`Basic.Monoid`里。我们也把`ListFunctor`注册成`Monoid`环境。

In [9]:
type(Functor.mappend(s,s))

Contexts.Functor.ListFunctor

In [10]:
Functor.mappend(s,s)

[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8]

In [11]:
Functor.mconcat(s.mrb([1,2]))

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

In [12]:
type(Functor.mconcat(s.mrb([1,2])))

Contexts.Functor.ListFunctor

In [13]:
Functor.mempty(s)

[]

In [14]:
type(Functor.mempty(s))

Contexts.Functor.ListFunctor

In [15]:
Functor.mempty(s).mrb([1,2])

[]

##### `Sigive`：信号范畴的Applicative环境
`Sigive`在`Sigor`的基础上加上了`ap`类方法。`ap`的功能是同时提升输入参数中每个算法并且对自身进行映射，将结果依次合并到同一容器中。输入参数必须是一个`list`的单参数函数序列。`Sigive`适合同构，独立性多循环操作。

以下案例是将`float`和`str`同时应用于自身并依次将结果合并成`sig_0`容器：

In [16]:
a=[x for x in range(1,9)]
s=Sigive(a)
s.ap([float,str])

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8']

In [17]:
a=np.arange(1,9)
s=Sigive(a)
s.ap([float,str])

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8']

以下两个案例演示`Sigive`也是`Sigor`:

In [18]:
s.fmap(float)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [19]:
type(s.fmap(float))

Contexts.Functor.ListFunctor

In [20]:
s.mrb([3,4])

[[3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4]]

`Sigive`还有其他三个法则，分别是`pure`,`then`和`liftA2`。`pure`将输入参数塞进`ListApplicative`，返回包含这个参数的一个`ListApplicative`：

In [21]:
s.pure(1)

[1]

`then`是将输入参数替换每个元素，与`mconcat.mrb`等价。输入必须是`list`（因为python对类的松散，只要是`Iterable`也是可以接受的）：

In [22]:
s.then([1,2])

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

In [23]:
Applicative.mconcat(s.mrb([1,2]))

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

In [24]:
s.then((1,2))

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]

`liftA2`的输入是二元函数和一个`list`对象。二元函数依次作用于自身的每个元素和输入`list`每个元素，也就是`liftA2`将输入的二元函数提升为一个`ListApplicative`的态射：

In [25]:
from operator import add
s.liftA2(add,[1])

[2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
s.liftA2(add,(1,2))

[2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]

`Contexts.Applicative`还提供了若干有用的函数：`liftA2(f:Callable[[A,B],C],xs:Iterable[A],ys:Iterable[B])->Generator[C,None,None]`, `@ap.register`, `@pure.register`。其中`@ap.register`和`@pure.register`都注册了`list`类：

In [27]:
Applicative.mappend(Applicative.pure([float]),Applicative.pure([str]))

[float, str]

In [28]:
Applicative.pure([1,2])

[1, 2]

In [29]:
type(Applicative.pure([1,2]))

Contexts.Applicative.ListApplicative

In [30]:
Applicative.pure([[1,2]])

[[1, 2]]

In [31]:
Applicative.ap([float,str],[1,2,3,4])

[1.0, 2.0, 3.0, 4.0, '1', '2', '3', '4']

以下案例拓展`Applicative.ap`可以接受`tuple`数据类型：

In [32]:
@Applicative.ap.register(tuple)
def _(a,b): return ListApplicative((f(x) for f in a for x in b))

In [33]:
Applicative.ap((float,str),(1,2,3,4))

[1.0, 2.0, 3.0, 4.0, '1', '2', '3', '4']

##### `SigMonad`：信号范畴的Monad环境
`SigMonad`在`Sigive`的基础上加上了`bind`的功能。`bind`的输入参数为函数`f:Callable[[A],SigMonad[B]]`，函数作用于`SigMonad`的元素，`bind`的功能就是将这个函数提升为`SigMonad`的态射。`bind`等价于`mconcate.fmap`，通过`mconcate`可以改变原来`SigMonad`的结构：

In [34]:
s=SigMonad(a)

In [35]:
s.bind(lambda x: [x-1,x,x+1])

[0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9]

In [36]:
s.fmap(lambda x: [x-1,x,x+1])

[[0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9]]

In [37]:
Monad.mconcat(s.fmap(lambda x: [x-1,x,x+1]))

[0, 1, 2, 1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9]

In [38]:
s.bind(lambda x: [x-1,x,x+1]).bind(lambda x: [x+1])

[1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7, 6, 7, 8, 7, 8, 9, 8, 9, 10]

接下来看看一个模拟国际象棋的案例，摘自于[list monad](http://learnyouahaskell.com/a-fistful-of-monads#the-list-monad)。将棋盘的8x8个格子用坐标$(x,y),\ x,y\in[1,2,3,4,5,6,7,8]$来标识。问骑士能否从位置$(6,2)$走三步移到$(6,1)$。

In [39]:
KinghtPos=tuple[int,int]

`moveKnight`计算骑士在给定的位置可以走动的所有坐标：

In [40]:
def moveKnight(p:KinghtPos)->list[KinghtPos]:
    board=[1,2,3,4,5,6,7,8]
    match p:
        case (c,r):
            return list(filter((lambda x: (x[0] in board) and (x[1] in board)),
                          [(c+2,r-1),(c+2,r+1),(c-2,r-1),(c-2,r+1),(c+1,r-2)
                           ,(c+1,r+2),(c-1,r-2),(c-1,r+2)]))
        case _: return []

骑士从位置$(6,2)$有六个可走到的坐标：

In [41]:
moveKnight((6,2))

[(8, 1), (8, 3), (4, 1), (4, 3), (7, 4), (5, 4)]

`in3`计算骑士走三步所有可能的坐标：

In [42]:
def in3(start:KinghtPos)->list[KinghtPos]:
    return Monad.pure([start]).bind(moveKnight).bind(moveKnight).bind(moveKnight)

- `Monad.pure([start])`：将`start`放进monad环境。
- 第一个`bind(moveKnight)`：将`moveKnight`应用于monad环境中的元素`start`，算出走第一步所有可能的位置，返回以这些坐标为元素的monad。
- 第二个`bind(moveKnight)`：将`moveKnight`依次应用于第一步所得的monad中的元素，算出第二步所有可以到达的坐标，同样返回以这些坐标为元素的monad。
- 第三个`bind(moveKnight)`：与第二步类似，得到第三步结果。

对骑士从坐标$(6,2)$出发走三步后的所有可能坐标（有些坐标有重复）如下：

In [43]:
in3((6,2))

[(8, 1),
 (8, 3),
 (4, 1),
 (4, 3),
 (7, 4),
 (5, 4),
 (5, 2),
 (5, 4),
 (8, 1),
 (8, 5),
 (6, 1),
 (6, 5),
 (8, 1),
 (8, 3),
 (4, 1),
 (4, 3),
 (7, 4),
 (5, 4),
 (8, 3),
 (8, 5),
 (4, 3),
 (4, 5),
 (7, 2),
 (7, 6),
 (5, 2),
 (5, 6),
 (5, 2),
 (8, 3),
 (6, 3),
 (5, 4),
 (5, 6),
 (8, 3),
 (8, 7),
 (6, 3),
 (6, 7),
 (8, 1),
 (8, 3),
 (4, 1),
 (4, 3),
 (7, 4),
 (5, 4),
 (4, 1),
 (4, 3),
 (3, 4),
 (1, 4),
 (7, 2),
 (7, 4),
 (3, 2),
 (3, 4),
 (6, 1),
 (6, 5),
 (4, 1),
 (4, 5),
 (5, 2),
 (5, 4),
 (1, 2),
 (1, 4),
 (4, 1),
 (4, 5),
 (2, 1),
 (2, 5),
 (8, 1),
 (8, 3),
 (4, 1),
 (4, 3),
 (7, 4),
 (5, 4),
 (8, 3),
 (8, 5),
 (4, 3),
 (4, 5),
 (7, 2),
 (7, 6),
 (5, 2),
 (5, 6),
 (4, 1),
 (4, 3),
 (3, 4),
 (1, 4),
 (4, 3),
 (4, 5),
 (3, 2),
 (3, 6),
 (1, 2),
 (1, 6),
 (7, 2),
 (3, 2),
 (6, 3),
 (4, 3),
 (7, 4),
 (7, 6),
 (3, 4),
 (3, 6),
 (6, 3),
 (6, 7),
 (4, 3),
 (4, 7),
 (5, 2),
 (1, 2),
 (4, 3),
 (2, 3),
 (5, 4),
 (5, 6),
 (1, 4),
 (1, 6),
 (4, 3),
 (4, 7),
 (2, 3),
 (2, 7),
 (7, 2),
 (7, 4),
 

`canReachIn3`计算骑士从`start`坐标开始，三步是否能到达`end`:

In [44]:
def canReachIn3(start:KinghtPos,end:KinghtPos)->bool:
    return end in in3(start)

算法很简单就是看`end`是否在走三步后所有可能的坐标里面。

答案是骑士从$(6,2)$出发，走三步可以到达$(6,1)$：

In [45]:
canReachIn3((6,2),(6,1))

True

#### 迭代环境

##### `SigFoldable`
`SigFoldable`有四个类方法，分别是`foldl`，`foldr`，`scanl`，`scanr`。假设容器的元素集合为$[x_1,x_2,\cdots,x_n]$，
- `foldl`：`(f:Callable[[B,A],B],x0:B)->B`，迭代计算$y_1=f(x_0,x_1),\ y_2=f(y_1,x_2),\cdots,\ y_n=f(y_{n-1},x_n)$，返回$y_n$，其中$f$为第一个输入参数，$x_0$为第二个输入参数。
- `foldr`：`(f:Callable[[A,B],B],x0:B)->B`，迭代计算$y_1=f(x_n,x_{n+1}),\ y_2=f(x_{n-1},y_1),\cdots,\ y_n=f(x_1,y_{n-1})$，返回$y_n$，其中$f$为第一个输入参数，$x_{n+1}$为第二个输入参数。
- `scanl`：`(f:Callable[[B,A],B],x0:B)->B`，迭代计算$y_1=f(x_0,x_1),\ y_2=f(y_1,x_2),\cdots,\ y_n=f(y_{n-1},x_n)$，返回$[y_1,y_2,\cdots,y_n]$，其中$f$为第一个输入参数，$x_0$为第二个输入参数。
- `scanr`：`(f:Callable[[A,B],B],x0:B)->B`，迭代计算$y_1=f(x_n,x_{n+1}),\ y_2=f(x_{n-1},y_1),\cdots,\ y_n=f(x_1,y_{n-1})$，返回$[y_1,y_2,\cdots,y_n]$，其中$f$为第一个输入参数，$x_{n+1}$为第二个输入参数。

In [46]:
s=SigFoldable(a)
print(s)

[1, 2, 3, 4, 5, 6, 7, 8]


In [47]:
type(a)

numpy.ndarray

In [48]:
s.foldl(add,0)

36

In [49]:
s.foldr(add,0)

36

In [50]:
s.scanl(add,0)

[1, 3, 6, 10, 15, 21, 28, 36]

In [51]:
s.scanr(add,0)

[8, 15, 21, 26, 30, 33, 35, 36]

In [72]:
a=[[1,2,3,4],[5,6,7,8]]
s=SigFoldable(a)

In [76]:
from functools import partial
def leftEdge(n,acc,x): return (acc[1][-n:],x)
def rightEdge(n,x,acc): return x+(acc[1][:n],)
r2=partial(rightEdge,2)
l2=partial(leftEdge,2)
v_le=([],[-1,0])
v_re=([],[9,10],[])

In [77]:
s.scanl(l2,v_le).scanr(r2,v_re)[::-1]

[([-1, 0], [1, 2, 3, 4], [5, 6]), ([3, 4], [5, 6, 7, 8], [9, 10])]

```{note}
`v_le`和`v_re`不仅是初始值，而且分别定义了输出容器元素的形状。
```

`Contexts.Foldable`提供了这四个函数，可用于`Iterable`类型：

In [52]:
from Basic.Monoid import mappend, mempty

v=[[1,2],[3,4]]
foo=Foldable.scanl(mappend,mempty(v),v)

In [53]:
for i in foo: print(i)

[1, 2]
[1, 2, 3, 4]


In [54]:
Foldable.foldl(mappend,mempty(v),v)

[1, 2, 3, 4]

In [55]:
v=((1,2),(3,4))
Foldable.foldl(mappend,mempty(v),v)

(1, 2, 3, 4)

In [56]:
Foldable.foldl(mappend,mempty(v),reversed(v))

(3, 4, 1, 2)

### 算法
#### 边界处理方法
#### 方法
边界处理方法包含零延伸，周期延伸和对称延伸三种。假设信号长度为$n$，延伸的长度为$m$，通常我们要求$m\leq n$，信号为$[x_1,x_2,\cdots,x_n]$。

顾名思义，零延伸就是对区间外用零值延伸。即左右延伸都是$[0,\cdots,0]$，延伸的数组长度为$m$。

周期延伸是将区间中的信号看成周期函数进行延伸，因此，左延伸数组为$[x_{n-m+1},\cdots,x_n]$，这是因为根据周期性，元素$x_1$往左跟元素$x_n$相邻。同理，右延伸数组为$[x_1,x_2,\cdots,x_m]$。

对称延伸是以端点为原点的对称延伸：左延伸数组为$[x_m,x_{m-1},\cdots,x_1]$, 右延伸数组为$[x_n,x_{n-1},\cdots,x_{n-m+1}]$

#### 程序设计
程序包括三个数据类型，三种算法函数。三种函数为`zero`，`periodic`，`symetric`分别与零延伸，周期延伸和对称延伸相对应。函数的接口：输入接口为`(n,v)`，其中`n`为延伸的长度，类型为`int`，`v`为被延伸的信号，类型为`sig_0`。函数要求`n`不大于信号的长度的正整数。输出接口为`Extensions`的数据类，这个数据类是一对`sig_0`，即`(sig_0,sig_0)`，其中左元素对应左延伸数组，右元素对应右延伸数组。为了方便使用，定义了另外两个数据类：
- `PaddingMethodNames`: 延伸方法名称，`Enum`数据类，其中的域值为`ZERO`,`PERIODIC`和`SYMETRIC`。
- `padding`: dictionary数据类，其中`key`为`PaddingMethoNames`，`value`为上面的延伸函数。

In [57]:
#from Signal.Padding import PaddingMethodNames, padding

from typing import NewType
from enum import Enum
from Signal.Types import sig_0

Extensions = NewType('Extensions',tuple[sig_0,sig_0])
PaddingMethodNames=Enum('PaddingMethod',['ZERO','PERIODIC','SYMETRIC'])

def zero(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions(([0.0]*n,[0.0]*n))

def periodic(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions((v[-n:],v[:n]))

def symetric(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions((v[:n][::-1],v[-n:][::-1]))

padding={name:method for (name,method) in zip(PaddingMethodNames,(zero,periodic,symetric))}

In [58]:
for i in PaddingMethodNames: print(i)

PaddingMethod.ZERO
PaddingMethod.PERIODIC
PaddingMethod.SYMETRIC


In [59]:
a=np.arange(1,9)
foo = map((lambda x: padding[x](4,a)), PaddingMethodNames)
print('原信号: ',a)
for i,j in zip(PaddingMethodNames,foo): print(str(i)+":",j)

原信号:  [1 2 3 4 5 6 7 8]
PaddingMethod.ZERO: ([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0])
PaddingMethod.PERIODIC: (array([5, 6, 7, 8]), array([1, 2, 3, 4]))
PaddingMethod.SYMETRIC: (array([4, 3, 2, 1]), array([8, 7, 6, 5]))


In [60]:
#from Signal.Utils import ispower2

def ispower2(n:int)->bool:
    return (n & (n-1) == 0) and n != 0